In [ ]:
#script for auxiliar functions.

In [3]:
#Aux functions
#def function prod computes the product 1^t*Z*t.
#@param: n : the dimension of the matrices.
function produ(Z,n)
    return(ones(n)'*Z*ones(n))
end

#def function roundMatrix applies the function roundZero to every element of the matrix.
#@param: A : matrix to round
function roundZeroMatrix(A)
    map(A) do x
        roundZero(x)
    end
end

#def function roundZero rounds close reals to zero to zero
function roundZero(a)
    if(abs(a)<=0.0001)
        return(0)
    end
    return(a) 
end

#def function nuclearNorm Computes the nuclear norm of a matrix
#@param mat : mat to compute the nuclear norm
function nuclearNorm(mat)
    return(sum(svdfact(mat)[:S]))
end

#function which computes the estimation of the error as by theorem 3.1
#@param delta: radius of the wasserstein metric ball.
#@param A : solution of the optimization problem.
#@param B : Array of the observation matrix
#@param N : numero de observaciones
#@param n : dimension de las matrices
function estimarCota(delta,A,B,N,n)
    a = delta*nuclearNorm(2*A-ones(n)*ones(n)')
    [B[i] = A-B[i] for i in 1:N]
    d = (1/N)*sum(norm1.(B))
    return(a+d)
end
#function that computes the norm1 of a matrix. note that the norm(,1) implemented
#in julia is the induced 1 norm.
#@param W: a matrix
function norm1(W)
    a= 0
    map(W) do x
        a = abs(x)+a
    end
    return(a)
end

#function to generate a random Erdos-renyi graph with a given number of 
# clusters and probabilities. Creates an inferior triangluar matrix and then sums its transpose.
#@param: probabilities: vector of probabilities>0.5 of each intracluster probability.
#The size of the vector is the number of clusters. min(probabilities)>1/2
#@param: sizes : the sizes of the clusters.
#@param: n : the dimension of the adjacency matrix.
#@param: q : outercluster adjacency probability : q<1/2
#@Pre: the sum of the sizes must be n.
function erdosgraph(probabilities,q,sizes,n)
    A= zeros(n,n)
    contador2=0
    proba = 1
    contador1=1
    for k in sizes
        contador2=contador2+k
        for j in contador1:contador2
            for i in j+1:n
                rand1=rand()
                rand2=rand()
                if(i>=contador1 && i<=contador2)
                    if(rand1<=probabilities[proba])
                        A[i,j]=1
                    end
                else
                    if(rand2<=q)
                        A[i,j]=1
                    end
                end
            end
        end
        proba=proba+1
        contador1=contador1+k
    end
    return(A+A')
end

function erdosgraph2(probabilities,q,sizes,n)
    A= zeros(n,n)
    for r in 1:n
        for w in r+1:n
            rand0 = rand()
            if(rand0<=q)
                A[r,w]=1
            end
        end
    end    
    for i in 1:sizes[1]-1
        for j in i+1: sizes[1]
            rand1=rand()
            if(rand1<=probabilities[1])
                A[i,j]=1
            else
                A[i,j]=0
            end
        end
    end
    for k in sizes[1]+1:sizes[1]+sizes[2]-1
        for l in k+1: sizes[1]+sizes[2]
            rand2=rand()
            if(rand2<=probabilities[2])
                A[k,l]=1
            else
                A[k,l]=0
            end
        end
    end
    return(A+A')
end

#Calcula la distancia en norma 1 de dos matrices
function distance1(A,B)
    return(vecnorm(roundZeroMatrix(A-B),1))
end

#Erdos graph function that fixes an r,s entry of the matrix to 1. This allows to test if 
# our method vs the mean method. 

function erdosgraph3(probabilities,q,sizes,n,r,s)
    A= zeros(n,n)
    contador2=0
    proba = 1
    contador1=1
    for k in sizes
        contador2=contador2+k
        for j in contador1:contador2
            for i in j+1:n
                rand1=rand()
                rand2=rand()
                if(i>=contador1 && i<=contador2)
                    if(rand1<=probabilities[proba])
                        A[i,j]=1
                    end
                else
                    if(rand2<=q)
                        A[i,j]=1
                    end
                end
            end
        end
        proba=proba+1
        contador1=contador1+k
    end
    adjacencia = A+A'
    adjacencia[r,s] = 1
    adjacencia[s,r] = 1

    return(adjacencia)
end



erdosgraph3 (generic function with 1 method)

<h1>Funciones de evaluacion de graph learning lineal<h1>

In [ ]:
# evalua el modelo variando la probabilidad intra cluster y outra cluster. cada simulacion toma unos 3 minutos
# y resuelve 2400 problemas de optimizacion aprox. Evaluar graph learning 3 no pone un 1 en las diagonales de las matrices.
function evaluarLinearGraphLearning3(delta,observaciones)
    real = erdosgraph([1,1,1],0,[5,3,4],12)
    lala=zeros(9,9)
    indicet=1
     for t in 1:-0.05:0.6
        indicew=1
        for w in 0:0.05:0.4
            contador=0
            for u in 1:50
                B=Array{Any}(observaciones)
                [B[r]=erdosgraph([t,1,1],w,[5,3,4],12) for r in 1:observaciones]
                respuesta=learnLinearGraph(ones(observaciones)/observaciones,delta,B,12)
                errorGrafo = distance1(real,respuesta[1])
                contador=contador+errorGrafo
            end
       # println(contador/50)
            lala[indicet,indicew]=contador/50
        indicew=indicew+1
        end
    indicet=indicet+1
    end
    #println(lala)
    return(lala)
end

In [ ]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein.
function evaluarLinearGraphLearning4(inProba,outProba,observaciones,intervalo=0.01:0.04:2.0)
    real = erdosgraph([1,1],0,[12,8],20)
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:10
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[12,8],20) for r in 1:observaciones]
            respuesta=learnLinearGraph(ones(observaciones)/observaciones,t,B,20)
                errorGrafo = distance1(real,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/10
            indicet=indicet+1
    end
    return([lala,intervalo])
end

In [ ]:
#evaluarLinearGraphLearning6 es una funcion que mide el error de recuperaciones al variar el numero de observaciones.
#el radio de la bola se varia a varios niveles. Se compara con el modelo de promedios.

function evaluarLinearGraphLearning6(inProba,outProba,delta)
    real = erdosgraph([1,1,1],0,[5,3,4],12)
       lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:25
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,1,1],outProba,[5,3,4],12) for r in 1:t]
            respuesta=learnLinearGraph(ones(t)/t,delta,B,12)
                errorGrafo = distance1(real,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/25
            indicet=indicet+1
    end
    return(lala)
end

In [ ]:
function evaluarGraphLearninglineal6grande(inProba,outProba,delta,redondeado)
    real = erdosgraph([1,1],0,[25,25],50)
       lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:5
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,inProba],outProba,[25,25],50) for r in 1:t]
            if(redondeado==true)
                respuesta=round.(learnLinearGraph(ones(t)/t,delta,B,50)[1],0)
            else
                respuesta=learnLinearGraph(ones(t)/t,delta,B,50)[1]
            end
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/5
            indicet=indicet+1
    end
    return(lala)
end

<h1>Funciones de evaluacion de graph learning espectral<h1>

In [ ]:
#evaluarGraphLearning6 es una funcion que mide el error de recuperaciones al variar el numero de observaciones.
#el radio de la bola se varia a varios niveles. Se compara con el modelo de promedios. No se ponen unos en la diagonas
# al igual que en evaluar6 lineal.

function evaluarGraphLearningEspectral6(inProba,outProba,delta,redondeado)
    real = erdosgraph([1,1,1],0,[5,3,4],12)+diagm(ones(12))
       lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:25
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,1,1],outProba,[5,3,4],12)+diagm(ones(12)) for r in 1:t]
            if(redondeado==true)
                respuesta=round.(learnEspectralGraph(ones(t)/t,delta,B,12)[1],0)
            else    
                respuesta=learnEspectralGraph(ones(t)/t,delta,B,12)[1]
            end
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/25
            indicet=indicet+1
    end
    return(lala)
end






In [ ]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein.
function evaluarspectralGraphLearning4(inProba,outProba,observaciones,intervalo=0.01:0.1:5.0)
    verdadera = erdosgraph([1,1],0,[12,8],20)+diagm(ones(20))
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:10
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[12,8],20)+diagm(ones(20)) for r in 1:observaciones]
            respuesta=learnEspectralGraph(ones(observaciones)/observaciones,t,B,20)
                errorGrafo = distance1(verdadera,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/10
            indicet=indicet+1
    end
    return([lala,intervalo])
end

<h1>Funciones de evaluacion de graph learning l2<h1>

In [ ]:
function evaluarGraphLearningl26(inProba,outProba,delta)
    real = erdosgraph([1,1,1],0,[5,3,4],12)+diagm(ones(12))
       lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:25
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,1,1],outProba,[5,3,4],12)+diagm(ones(12)) for r in 1:t]
                respuesta=learnl2Graph(ones(t)/t,delta,B,12)
                errorGrafo = distance1(real,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/25
            indicet=indicet+1
    end
    return(lala)
end

In [ ]:
function evaluarGraphLearningl262(inProba,outProba,delta,redondeado)
    real = erdosgraph([1,1,1],0,[5,3,4],12)
       lala= Array{Any}(40)
    indicet=1
    for t in 1:40
            contador=0
            for u in 1:25
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,1,1],outProba,[5,3,4],12) for r in 1:t]
            if(redondeado==true)
                respuesta=round.(learnl2Graph(ones(t)/t,delta,B,12)[1],0)
            else
                respuesta=learnl2Graph(ones(t)/t,delta,B,12)[1]
            end
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/25
            indicet=indicet+1
    end
    return(lala)
end

In [ ]:
# evalua el modelo variando la probabilidad intra cluster y outra cluster. cada simulacion toma unos 3 minutos
# y resuelve 2400 problemas de optimizacion aprox. Evaluar graph learning 3 no pone un 1 en las diagonales de las matrices.
function evaluarl2GraphLearning3(delta,observaciones)
    real = erdosgraph([1,1],0,[10,10],20)
    lala=zeros(9,9)
    indicet=1
     for t in 1:-0.05:0.6
        indicew=1
        for w in 0:0.05:0.4
            contador=0
            for u in 1:50
                B=Array{Any}(observaciones)
                [B[r]=erdosgraph([t,t],w,[10,10],20) for r in 1:observaciones]
                respuesta=learnl2Graph(ones(observaciones)/observaciones,delta,B,20)
                errorGrafo = distance1(real,respuesta[1])
                contador=contador+errorGrafo
            end
       # println(contador/50)
            lala[indicet,indicew]=contador/50
        indicew=indicew+1
        end
    indicet=indicet+1
    end
    #println(lala)
    return(lala)
end

In [ ]:
function evaluarGraphLearningl26grande(inProba,outProba,delta,redondeado)
    real = erdosgraph([1,1],0,[25,25],50)
       lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:5
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,inProba],outProba,[25,25],50) for r in 1:t]
            if(redondeado==true)
                respuesta=round.(learnl2Graph(ones(t)/t,delta,B,50)[1],0)
            else
                respuesta=learnl2Graph(ones(t)/t,delta,B,50)[1]
            end
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/5
            indicet=indicet+1
    end
    return(lala)
end

In [ ]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein.
function evaluarl2GraphLearning4(inProba,outProba,observaciones,intervalo=0.01:0.1:5.0)
    verdadera = erdosgraph([1,1],0,[12,8],20)
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:10
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[12,8],20) for r in 1:observaciones]
            respuesta=learnl2Graph(ones(observaciones)/observaciones,t,B,20)
                errorGrafo = distance1(verdadera,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/10
            indicet=indicet+1
    end
    return([lala,intervalo])
end

<h1>Funciones de evaluacion para grafo promedio<h1>

In [ ]:
#method for matrix recovery using the mean of the observations. 
#B the observations to recover the graph
#n dimension of the adjacency matrices
#N number of observations

function meanGraphLearn(B)
    N = length(B)
    A= sum(B)/N
    return A
end
    

#method for matrix recovery using the mean of the observations. Returns the rounded result. 
#B the observations to recover the graph
#n dimension of the adjacency matrices
#N number of observations

function meanGraphLearn(B,redondeado)
    N = length(B)
    A= sum(B)/N
    if(redondeado==true)
        return round.(A,0)
    else
        return(A)
    end
end
    

In [6]:
#Evalua la tasa de decaimiento del error del metodo meangraphlearn al aumentar el numero de observaciones. 
function evaluarGraphLearningMean6(inProba,outProba,redondeado)
    real = erdosgraph([1,1,1],0,[5,3,4],12)
    lala= Array{Any}(50)
    indicet=1
    for t in 1:50
            contador=0
            for u in 1:50
                B=Array{Any}(t)
                [B[r]=erdosgraph([inProba,1,1],outProba,[5,3,4],12) for r in 1:t]
                respuesta=meanGraphLearn(B,redondeado)
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/50
            indicet=indicet+1
    end
    return(lala)
end

evaluarGraphLearningMean6 (generic function with 1 method)

In [ ]:
#Evalua la tasa de decaimiento del error del metodo meangraphlearn al aumentar el numero de observaciones. 
function evaluarGraphLearningMean6Grande(inProba,outProba,redondeado)
    real = erdosgraph([1,1],0,[25,25],50)
    lala= Array{Any}(30)
    indicet=1
    for t in 1:30
            contador=0
            for u in 1:50
                B=Array{Any}(t)
            [B[r]=erdosgraph([inProba,inProba],outProba,[25,25],50) for r in 1:t]
                respuesta=meanGraphLearn(B,redondeado)
                errorGrafo = distance1(real,respuesta)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/50
            indicet=indicet+1
    end
    return(lala)
end

<h1>Funciones de evaluacion para recuperacion por regularizacion<h1>

In [ ]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein usando la recuperacion regularizada por 2A-11^t.
function evaluarRegularized2AGraphLearning4(inProba,outProba,observaciones,intervalo=0.01:0.1:5.0)
    verdadera = erdosgraph([1,1],0,[12,8],20)+diagm(ones(20))
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:10
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[12,8],20)+diagm(ones(20)) for r in 1:observaciones]
            respuesta=learnRegularized2A(ones(observaciones)/observaciones,t,B,20)
                errorGrafo = distance1(verdadera,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/10
            indicet=indicet+1
    end
    return([lala,intervalo])
end

In [ ]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein usando la recuperacion regularizada por A.
function evaluarRegularizedAGraphLearning4(inProba,outProba,observaciones,intervalo=0.01:0.1:5.0)
    verdadera = erdosgraph([1,1],0,[12,8],20)+diagm(ones(20))
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:10
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[12,8],20)+diagm(ones(20)) for r in 1:observaciones]
            respuesta=learnRegularizedA(ones(observaciones)/observaciones,t,B,20)
                errorGrafo = distance1(verdadera,respuesta[1])
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/10
            indicet=indicet+1
    end
    return([lala,intervalo])
end

<h1>Funciones de evaluacion para grafos recuperados con ADMM recovery<h1>

In [3]:
#Evaluar graph learning 4 permite variar delta y encontrar errores promedio al ir moviendo el radio de la bola de la metrica
#de Wasserstein.
function evaluarADMMGraphLearning4(inProba,outProba,observaciones,intervalo=0:1:20.0)
    verdadera = erdosgraph([1,1],0,[40,160],200)+diagm(ones(200))
    lala=Array{Any}(length(intervalo))
    indicet=1
    for t in intervalo
            contador=0
            for u in 1:3
                B=Array{Any}(observaciones)
            [B[r]=erdosgraph([inProba,inProba],outProba,[40,160],200)+diagm(ones(200)) for r in 1:observaciones]
            respuesta=admmRecovery(B,t,200,1.6,1.0e-4,250)
            bli =(respuesta[1]+ones(200)*ones(200)')/2
            errorGrafo = distance1(verdadera,bli)
                contador=contador+errorGrafo
            end
            lala[indicet]=contador/3
            indicet=indicet+1
    end
    return([lala,intervalo])
end

evaluarADMMGraphLearning4 (generic function with 2 methods)